# PTT Crawler

   # 爬蟲架構

In [ ]:
#上層爬蟲:處理換頁、資料總筆數問題

In [ ]:
#中層爬蟲:處理單一頁面內每筆資料頁面(URL)連結問題

In [ ]:
#下層爬蟲:清理資料，放入DIST。

In [25]:
import requests
from bs4 import BeautifulSoup
from collections import OrderedDict
import json
import traceback
import re

In [2]:
HOST = 'https://www.ptt.cc'

In [23]:
list_ptt = []

#def ptt_crawler(no_pages):
def ptt_crawler(board_name, no_pages):
    res = requests.get(HOST + "/bbs/" + board_name + "/index.html")
    soup = BeautifulSoup(res.text, 'lxml')
    total_page = int(soup.select('a.wide')[1]['href'].split('index')[1].split('.html')[0]) + 1

    for page in range(total_page, total_page - no_pages, -1):
        url = HOST + "/bbs/{}/index{}.html".format(board_name,page)
        print(url)
        #交給中層爬蟲
        list_crawler(url)
    print("[INFO] crawl_ptt complete!")
    
    #將爬取資料寫到txt檔案
    crawler_writer(board_name)
    #將爬取資料寫到json檔案
    crawler_json_writer(board_name)
    
#中層爬蟲   
def list_crawler(url):
    res_ls = requests.get(url)
    soup_ls = BeautifulSoup(res_ls.text, 'lxml')
    for tag in soup_ls.select('div.title > a'):
        title = tag.text
        url_ls = HOST + tag['href']
        print(url_ls)
        #交給下層爬蟲
        article_crawler(url_ls)
    print("[INFO] {} complete!".format(url_ls))

#下層爬蟲
def article_crawler(url_ls):    
    res_article = requests.get(url_ls)
    soup_article = BeautifulSoup(res_article.text, 'lxml')
    article_dict = OrderedDict()
    
    meta_values = soup_article.select('span.article-meta-value')
    try:
        article_dict['author'] = meta_values[0].text
        article_dict['board'] = meta_values[1].text
        article_dict['title'] = meta_values[2].text
        article_dict['tm'] = meta_values[3].text
    except IndexError:
        print("[Error] Error while parsing meta_values")

    #push的資料判斷有問題，不確定if...else判斷式哪裡有錯
    #請老師告訴我語法哪裡有問題
    #謝謝!
    pushs = []
    try:
        for push in soup_article.select('div.push'):
            push_dict = {}
            push_spans = push.select('span')
            if push_spans[0].text != "":
                push_dict['push_type'] = push_spans[0].text.strip()
            else:
                push_dict['push_type'] = None
            if push_spans[1].text != "":
                push_dict['push_author'] = push_spans[1].text.strip()
            else:
                push_dict['push_author'] = None
            print(push_spans[2])
            if push_spans[2].text != ":":
                 push_dict['push_content'] = push_spans[2].text.split(': ')[1].strip()
            else:
                push_dict['push_content'] = None 
            if push_spans[3].text != "":
                push_dict['push_tm'] = push_spans[3].text.strip()
            else:
                push_dict['push_tm'] = None
            pushs.append(push_dict)

                 #將推文加入字典檔
            article_dict['pushs'] = pushs

        for tag in soup_article.select('span.article-meta-value'):
            tag.extract()
        for tag in soup_article.select('span.article-meta-tag'):
            tag.extract()
        for tag in soup_article.select('span.f2'):
            tag.extract()
        for tag in soup_article.select('div.push'):
            tag.extract()
        article_dict['content'] = soup_article.select_one("#main-content").text.strip()    
        print("[INFO] {} complete!".format(url_ls))
    except IndexError:
        print("[Error] Error while parsing push")
        print(traceback.format_exc())
        
    list_ptt.append(article_dict)
    
def crawler_writer(board_name):
    with open('./list_ptt_{}.txt'.format(board_name), 'w', encoding='utf-8') as f:
        for posts in list_ptt:
            f.write('\n')
            for keys in posts:
                valus_str = "{} : {}".format(keys, str(posts[keys]))
                f.write(valus_str + '\n')
    print("[INFO] riter complete!")

def crawler_json_writer(board_name):
    with open('./json_ptt_{}.json'.format(board_name), 'a', encoding='utf-8') as fj:
        fj.write(json.dumps(list_ptt, ensure_ascii=False))
    print("[INFO] json_writer complete!")

In [24]:
ptt_crawler('ObataTakeshi',2)

https://www.ptt.cc/bbs/ObataTakeshi/index52.html
https://www.ptt.cc/bbs/ObataTakeshi/M.1456976001.A.3D9.html
<span class="f3 push-content">: 覺得近期老師合作的作品節奏都有點怪…這篇鉑金也是</span>
<span class="f3 push-content">: 怎麼辦覺得不太好看...</span>
[INFO] https://www.ptt.cc/bbs/ObataTakeshi/M.1456976001.A.3D9.html complete!
https://www.ptt.cc/bbs/ObataTakeshi/M.1459527591.A.A71.html
<span class="f3 push-content">: 等好久啦～～～～～～～～～～～</span>
<span class="f3 push-content">: QAQ</span>
<span class="f3 push-content">: thanks!</span>
[INFO] https://www.ptt.cc/bbs/ObataTakeshi/M.1459527591.A.A71.html complete!
https://www.ptt.cc/bbs/ObataTakeshi/M.1462733392.A.44C.html
<span class="f3 push-content">: thanks!</span>
[INFO] https://www.ptt.cc/bbs/ObataTakeshi/M.1462733392.A.44C.html complete!
https://www.ptt.cc/bbs/ObataTakeshi/M.1464874549.A.080.html
<span class="f3 push-content">: 等了一個月　結果有一半都是明日在窩囊！！！！！！！！</span>
<span class="f3 push-content">: 還是覺得重點在主角完全沒有個人魅力...</span>
<span class="f3 push-content">: 更糟糕的是其他推動劇情的角色